In [1]:
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import os
import json
import unicodedata

In [5]:
state_dict = {
    "acre": "AC",
    "alagoas": "AL",
    "amapa": "AP",
    "amazonas": "AM",
    "bahia": "BA",
    "ceara": "CE",
    "distrito federal": "DF",
    "espirito santo": "ES",
    "goias": "GO",
    "maranhao": "MA",
    "mato grosso": "MT",
    "mato grosso do sul": "MS",
    "minas gerais": "MG",
    "para": "PA",
    "paraiba": "PB",
    "parana": "PR",
    "pernambuco": "PE",
    "piaui": "PI",
    "rio de janeiro": "RJ",
    "rio grande do norte": "RN",
    "rio grande do sul": "RS",
    "rondonia": "RO",
    "roraima": "RR",
    "santa catarina": "SC",
    "sao paulo": "SP",
    "sergipe": "SE",
    "tocantins": "TO"
}

In [6]:
def join_csv_files(folder_path):
  all_csv_files = []
  for root, _, files in os.walk(folder_path):
    for file in files:
      if file.endswith(".csv"):
        file_path = os.path.join(root, file)
        all_csv_files.append(file_path)

  dataframes = []
  for file in all_csv_files:
    dataframes.append(pd.read_csv(file))

  final_df = pd.concat(dataframes, ignore_index=True)
  return final_df

folder_path = "./data/focos"
combined_df = join_csv_files(folder_path)

combined_df.head()

,latitude,longitude,data_pas,satelite,pais,estado,municipio,bioma,numero_dias_sem_chuva,precipitacao,risco_fogo,id_area_industrial,frp
0,2.46000,-61.19000,2018-03-01 23:05:45,GOES-16,Brasil,RORAIMA,MUCAJAÍ,Amazônia,0.0,0.7,0.9,0,NaN
1,-29.86929,-51.55051,2018-06-09 16:54:00,NPP-375,Brasil,RIO GRANDE DO SUL,TRIUNFO,Pampa,2.0,0.0,0.8,0,3.0
2,-13.16900,-59.93400,2018-06-09 14:20:00,TERRA_M-T,Brasil,MATO GROSSO,COMODORO,Amazônia,7.0,0.0,0.7,0,8.2
3,-13.17000,-59.92500,2018-06-09 14:20:00,TERRA_M-T,Brasil,MATO GROSSO,COMODORO,Amazônia,7.0,0.0,0.6,0,12.8
4,-13.57400,-56.87600,2018-06-09 14:20:00,TERRA_M-T,Brasil,MATO GROSSO,SÃO JOSÉ DO RIO CLARO,Cerrado,18.0,0.0,1.0,0,14.4


In [7]:
combined_df['municipio'].nunique()

5298

In [9]:
def remove_accents(text):

    normalized_text = unicodedata.normalize('NFKD', text)
    return ''.join(ch for ch in normalized_text if not unicodedata.combining(ch))

In [10]:
combined_df['estado'] = combined_df['estado'].apply(lambda x: state_dict[remove_accents(x.lower())])

In [13]:
print(combined_df['estado'].nunique())
print(len(state_dict))

27
27


In [14]:
combined_df.to_csv('./data/focos/combined_focos.csv',index=False)

In [25]:
grouped_df = combined_df.groupby(['latitude', 'longitude', 'municipio'])

In [27]:
def get_highest_frp(group):
  return group.sort_values(by='frp', ascending=False).iloc[0]

In [28]:
filtered_df = grouped_df.apply(get_highest_frp)

KeyboardInterrupt: 

In [12]:
pd.read_csv('./data/rainforest_degradation/def_area_2004_2019.csv')
pd.read_csv('./data/rainforest_degradation/el_nino_la_nina_1999_2019.csv')
pd.read_csv('./data/rainforest_degradation/inpe_brazilian_amazon_fires_1999_2019.csv')

,year,month,state,latitude,longitude,firespots
0,1999,1,AMAZONAS,-2.371113,-59.899933,3
1,1999,1,MARANHAO,-2.257395,-45.487831,36
2,1999,1,MATO GROSSO,-12.660633,-55.057989,18
3,1999,1,PARA,-2.474820,-48.546967,87
4,1999,1,RONDONIA,-12.861700,-60.513100,1
...,...,...,...,...,...,...
2099,2019,12,MATO GROSSO,-12.138385,-56.066297,148
2100,2019,12,PARA,-2.976727,-50.909316,1280
2101,2019,12,RONDONIA,-10.809262,-62.890452,42
2102,2019,12,RORAIMA,2.856667,-60.431963,27


In [36]:
amazon = pd.read_csv('./data/forest_fire/amazon.csv', encoding='latin1')

In [37]:
with open('./data/state_dict.json', 'r') as file:
    state_dict = json.load(file)

In [39]:
amazon['state'] = amazon['state'].apply(lambda x: state_dict[x])
amazon.head()

,year,state,month,number,date
0,1998,AC,Janeiro,0.0,1998-01-01
1,1999,AC,Janeiro,0.0,1999-01-01
2,2000,AC,Janeiro,0.0,2000-01-01
3,2001,AC,Janeiro,0.0,2001-01-01
4,2002,AC,Janeiro,0.0,2002-01-01
